# Track Plays
> Tracking track plays over time.

- toc: true 
- badges: true
- comments: false
- categories: [asot, tracks]
- image: images/track-plays.png

In [3]:
#hide
import os
import yaml
import spotipy
import json
import altair as alt
import numpy as np
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

asot_radio_id = '25mFVpuABa9GkGcj9eOPce'

albums = []
results = sp.artist_albums(asot_radio_id, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
seen = set()  # to avoid dups
for album in albums:
    name = album['name']
    if name not in seen:
        seen.add(name)

singles = []
results = sp.artist_albums(asot_radio_id, album_type='single')
singles.extend(results['items'])
while results['next']:
    results = sp.next(results)
    singles.extend(results['items'])
seen = set()  # to avoid dups
for single in singles:
    name = single['name']
    if name not in seen:
        seen.add(name)

episodes = singles + albums

episodes.sort(key=lambda x: x['release_date']) # Sort by release date

## Introduction

How many unique tracks have been played on A State of Trance over the years? What's been played the most?

In this post, we'll examine track plays over time.

## Getting Started

First, some baseline analysis. Let's first figure out how many tracks played on A State of Trance are available on Spotify:

In [4]:
tracks_counted = 0

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                tracks_counted += 1
    except:
        pass

print(tracks_counted)

17369


Wow - 17,000+ total tracks have been played! Remember, as we learned in the ["Methodology" post](https://scottbrenner.github.io/asot-jupyter/asot/bpm/2020/04/27/methodology.html) some episodes - especially early ones - are incomplete.

How many unique tracks? Get your regular expressions ready ..

In [5]:
import re

unique_tracks = set()

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                unique_tracks.add(re.sub("[\(\[].*?[\)\]]", "", track['name'])) # Remove episode numbers from track names (ex. 'Eternity [ASOT 005]')
    except:
        pass

print(len(unique_tracks))

12722


As always, this is a "best guess" - an approximation.

## Calculating

Let's crunch some numbers.

Which tracks have the most plays?

In [6]:
from collections import defaultdict

tracks_counter = defaultdict(int)

for episode in episodes:
    try:
        for track in sp.album_tracks(episode['uri'])['items']:
            if "a state of trance" in track['name'].lower() or "- interview" in track['name'].lower():
                continue
            else:
                track_artist = track['artists'][0]['name']
                if len(track['artists']) > 1:
                    for artist in track['artists'][1:]:
                        track_artist += " & " + artist['name']
                tracks_counter[track_artist + ' - ' + re.sub("[\(\[].*?[\)\]]", "", track['name'])] += 1
    except:
        pass

top_tracks = sorted(tracks_counter.items(), key=lambda k_v: k_v[1], reverse=True)

## Results

Let's see what we've got!

In [93]:
for track in top_tracks[:25]:
    print(track)

('Rising Star - Clear Blue Moon  - Original Mix', 10)
('Gareth Emery & STANDERWICK & HALIENE - Saving Light ', 10)
('Solid Globe - North Pole  - Original Mix', 9)
('Active Sight - The Search For Freedom  - Original Mix', 9)
('EnMass - CQ  - Original Mix', 9)
('Jonas Steur - Castamara  - Original Mix', 9)
('Marco V - Simulated  - Original Mix', 8)
('Rank 1 - Awakening  - Original Mix', 8)
('Solarstone & Scott Bond - 3rd Earth  - Original Mix', 8)
('Pulser - My Religion  - Original Mix', 8)
('Paul van Dyk & Hemstock & Jennings - Nothing But You  - Original Mix', 8)
("Perry O'Neil - Wave Force  - Original Mix", 8)
('Re:Locate - Rogue  - Original Mix', 8)
('Jose Amnesia & Jennifer Rene - Louder  - Original Mix', 8)
("Armin van Buuren & Ana Criado - I'll Listen  - Original Mix", 8)
('ID - ID ', 8)
('RAM - RAMexico ', 8)
('Cosmic Gate & Forêt - Need To Feel Loved ', 8)
('Solarstone - Seven Cities  - Armin van Buuren Remix', 7)
('Solarstone & Scott Bond - Naked Angel  - Original Mix', 7)
('Ro

In a crude graph:

In [94]:
source = pd.DataFrame.from_dict(top_tracks[:25])

bars = alt.Chart(source).mark_bar().encode(
    x=alt.X('1:Q', title='Plays'),
    y=alt.Y('0:N', sort='-x', title='Track', axis=alt.Axis(labels=False)),
).properties(
    title="A State of Trance - Most-played tracks",
    width=500
)

text = alt.Chart(source).mark_text(dx=-350, color='white', align='left').encode(
    x=alt.X('1:Q', title='Plays'),
    y=alt.Y('0:N', sort='-x'),
    text=alt.Text('0:N')
)

bars + text

alt.LayerChart(...)